In [9]:
# !pip install google-search-results
# !pip install numexpr
# !pip install langchainhub

  Using cached langchainhub-0.1.14-py3-none-any.whl.metadata (478 bytes)
  Using cached types_requests-2.31.0.10-py3-none-any.whl.metadata (1.8 kB)
Using cached langchainhub-0.1.14-py3-none-any.whl (3.4 kB)
Using cached types_requests-2.31.0.10-py3-none-any.whl (14 kB)


Let's take a look at a simple example:

In [13]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, AgentExecutor

llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

llm_with_stop = llm.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
I need to find out who Leo DiCaprio's girlfriend is and then calculate her current age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"Vittoria CerettiNow I need to find out her current age.
Action: Search
Action Input: "Vittoria Ceretti age"25 yearsNow I can calculate her current age raised to the 0.43 power.
Action: Calculator
Action Input: 25^0.43Answer: 3.991298452658078I now know the final answer.
Final Answer: Vittoria Ceretti's current age raised to the 0.43 power is approximately 3.99.

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': "Vittoria Ceretti's current age raised to the 0.43 power is approximately 3.99."}

• ```llm = ChatOpenAI(temperature=0)```: Create an instance of the ChatOpenAI language model with a temperature of 0.

• ```tools = load_tools(["serpapi", "llm-math"], llm=llm)```: Load the tools "serpapi" and "llm-math" with the specified language model.

• ```prompt = hub.pull("hwchase17/react")```: Retrieve the prompt from the hub with the name "hwchase17/react".

• ```prompt = prompt.partial(tools=render_text_description(tools), tool_names=", ".join([t.name for t in tools]))```: Modify the prompt by rendering the text description of the tools and joining their names.

• ```llm_with_stop = llm.bind(stop=["\nObservation"])```: Bind the language model with a stop condition set to the token "\nObservation".

• ```agent = (...)```: Define the agent using a series of steps, including input mapping, prompt, language model with stop condition, and output parsing.

• ```agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)```: Create an instance of the AgentExecutor with the defined agent and tools.

• ```agent_executor.invoke(...)```: Invoke the agent by passing an input to it and execute its chain of actions and tools.